In [2]:
# dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-31 07:37:34--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2024-10-31 07:37:34 (14.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
print("The length of dataset: ", len(text))

The length of dataset:  1115394


In [5]:
sample = text[100:1000]
print(sample)

 are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.




In [6]:
a = "hello"
c = set(a)
c

{'e', 'h', 'l', 'o'}

In [7]:
sorted(list(c))

['e', 'h', 'l', 'o']

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size) # our vocabulary for character level language model


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# character to integer for mapping vocabulary foreach.

stoi = { ch:i for i, ch in enumerate(chars)} # character to index
itos = { i:ch for i, ch in enumerate(chars)} # index to character

In [10]:
stoi

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [11]:
encode = lambda s: [stoi[c] for c in s] # take a string output a list of integer
decode = lambda i: "".join([itos[c] for c in i]) # take a list of integer output a string

print(encode("what is AI?"))
print(decode(encode("What is AI?")))

[61, 46, 39, 58, 1, 47, 57, 1, 13, 21, 12]
What is AI?


In [12]:
# encoding the entire text dataset
import torch

data = torch.tensor(encode(text), dtype=torch.long) # all data encoded to its integer value
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [13]:
len(text), data.shape # equal no of characters and integer after encoding the text

(1115394, torch.Size([1115394]))

In [14]:
# split data into train and validation set

n = int(0.9 * len(data)) # 90 percent of our data for train
train_data = data[:n]
val_data = data[n:]

In [15]:
decode(train_data[:200].tolist())

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you'

In [16]:
len(train_data), len(val_data)

(1003854, 111540)

In [17]:
# we dont send all data at once to the model, we send in chunk as block_size

block_size = 8
train_data[:block_size+1] # first nine character in dataset

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [18]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [19]:
# concept for later
ix_ = torch.randint(10, (3, )) # get the random 3 integer from 0 to 9, 3 is batch size
ix_ # random index from 0 to 9 for our 2 block size batch so that if we get ix_ = 9 we can reach to index 11

tensor([6, 1, 7])

In [20]:
# concept for later
numbers = torch.arange(0, 12) # 11 number list
x_ = torch.stack([numbers[i:i+2] for i in ix_]) # 3 is block size so
x_ # here we get a data of 2 block size and 3 batch size

tensor([[6, 7],
        [1, 2],
        [7, 8]])

In [21]:
torch.manual_seed(1337)
batch_size = 4 # how many sequence to process at one time
block_size = 8 # maximum context length for predictions

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, )) # get random 8 integer from 0 to len(data) - 8
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

In [22]:
xb, yb = get_batch("train")
print(f"Inputs: {xb.shape}")
print(xb)
print(f"Targets: {yb.shape}")
print(yb)

Inputs: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [23]:
yb[0], yb[0,0]

(tensor([43, 58,  5, 57,  1, 46, 43, 39]), tensor(43))

In [24]:
xb[0], yb[0]

(tensor([24, 43, 58,  5, 57,  1, 46, 43]),
 tensor([43, 58,  5, 57,  1, 46, 43, 39]))

In [25]:
for b in range(batch_size): # iterate 4 times as size of batch
  print(f"Start of batch {b}_____________________________________________")
  for t in range(block_size): # in each size iterate 8 time as for each item in a block size 8 for each
    context = xb[b, :t+1] #
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")
  print(f"End of batch {b}_______________________________________________")
  if b < batch_size - 1 : print(f"ANOTHER BATCH {b+1}\n\n")

Start of batch 0_____________________________________________
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
End of batch 0_______________________________________________
ANOTHER BATCH 1


Start of batch 1_____________________________________________
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
End of batch 1_______________________________

In [26]:
print(xb) # input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [27]:
# concept: tensor view
tensor = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12])
tensor.view(2,6), tensor.view(3,4), tensor.view(2,3,2)

(tensor([[ 1,  2,  3,  4,  5,  6],
         [ 7,  8,  9, 10, 11, 12]]),
 tensor([[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]]),
 tensor([[[ 1,  2],
          [ 3,  4],
          [ 5,  6]],
 
         [[ 7,  8],
          [ 9, 10],
          [11, 12]]]))

In [28]:
# concept: torch embedding
# (10, 3) # we have 10 words which make 3 dimension vector
embedding = torch.nn.Embedding(10, 3) # 10 tensor (num_embedding) of size 3 dimension vector
input = torch.LongTensor([[1,2,3,4],[5,6,7,8]]) # a batch of 2 samples of 4 indices each
output = embedding(input) # input (2,4) give output (2,4,3)
print(f"The shape of embedding: {output.shape}")
print(output)

The shape of embedding: torch.Size([2, 4, 3])
tensor([[[ 0.0643, -0.5024, -0.2026],
         [-1.5671, -1.0980,  0.2360],
         [-0.2398, -0.9211,  1.5433],
         [-0.3676, -0.7483,  1.0101]],

        [[ 0.1215,  0.1584,  1.1340],
         [-1.1539, -0.2984,  1.1490],
         [ 0.1812,  0.5467, -1.4948],
         [-1.2057,  0.5718, -0.5974]]], grad_fn=<EmbeddingBackward0>)


In [29]:
# bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # 65 vocab, 65 dim embedding

  def forward(self, idx, targets=None):
    # get idx: xbatch targets: ybatch
    logits = self.token_embedding_table(idx) # input is (4, 8) input batch
    # print(logits.shape) # (4, 8, 65) B, T, C

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      # reshape
      logits = logits.view(B*T, C) # (4 * 8, 65) = (32, 65)
      targets = targets.view(B*T) # (4 * 8) = (32)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array in the current context
    for _ in range(max_new_tokens):
      # get the prediction
      logits, loss = self(idx)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
    # return the idx after generation
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss) # 4 batch 8 time 65 dim embedding

torch.Size([32, 65]) tensor(4.8466, grad_fn=<NllLossBackward0>)


In [30]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


S'Nal3qizJ-OA$Oe-rTk.Mkv jgCSMCLzAiW?.uxtD! A gsSHvdRe!KzGXeJydQySrFmERqtu::wWvz.WxbjrQDme:a;ye;?K3$


In [31]:
idx = torch.randint(0, 65, (2,3), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

HVdSaGWZbK-x,sOeKXwEZfY;&wyqDml:B$tR$Lm;?rz.UK&!UG;v,pFZQye-hlkvuxtjqgoKXRejeec &sl
-$VXKs.fRecsJQ!zSg'


In [32]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [35]:
batch_size = 32

for step in range(10000):
  xb, yb = get_batch("train")
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  if step % 1000 == 0:
    print(f"Loss for step {step}: {loss}")

Loss for step 0: 2.4559085369110107
Loss for step 1000: 2.461034059524536
Loss for step 2000: 2.4624783992767334
Loss for step 3000: 2.545229434967041
Loss for step 4000: 2.487867593765259
Loss for step 5000: 2.5230562686920166
Loss for step 6000: 2.3995728492736816
Loss for step 7000: 2.4342026710510254
Loss for step 8000: 2.460646152496338
Loss for step 9000: 2.4901046752929688


In [40]:
generated_token = m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)
encoded_token = generated_token[0].tolist()
len(encoded_token)

101

In [41]:
decoded_token = decode(encoded_token)
print(decoded_token)


MEELo:
m,-thourord:
May f ond ga!
KEE y tibe t s gart hy f, ik migh my, mest t pesequrvind ROroo gs,


# self attention

In [43]:
B, T, C = 4, 8, 3 # batch size of 4 with block size 8 and 3 dimension
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 3])

In [45]:
a = torch.ones(3,3)
b = torch.randint(0,10, (3, 2)).float()
c = a @ b
print("Matrix multiply")
print(a)
print(b)
print(c)

Matrix multiply
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 6.],
        [9., 9.],
        [2., 6.]])
tensor([[12., 21.],
        [12., 21.],
        [12., 21.]])


In [47]:
torch.tril(torch.ones(3,3)) # lower traingular matrix of one

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [51]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10, (3, 2)).float()
c = a @ b
print("Matrix multiply")
print(a)
print(b)
print(c)

Matrix multiply
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 1.],
        [7., 9.],
        [4., 1.]])
tensor([[2.0000, 1.0000],
        [4.5000, 5.0000],
        [4.3333, 3.6667]])


In [55]:
wei = torch.tril(torch.ones(T, T)) # (8, 8)
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) -> (B, T, T) . (B, T , C) --> (B, T, C)

xbow2.shape, xbow2[0] # (1, 8, 8) (4, 8, 3)

(torch.Size([4, 8, 3]),
 tensor([[-1.2019, -0.3615,  1.2446],
         [-0.8425, -0.1719,  0.5324],
         [-0.4917,  0.1592, -0.1320],
         [-0.4021,  0.4558, -0.0859],
         [-0.5007,  0.2958, -0.2570],
         [-0.5120,  0.0888, -0.1345],
         [-0.2618, -0.0840, -0.2577],
         [-0.3937, -0.2030,  0.0299]]))

In [65]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [66]:
wei = torch.zeros((T,T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [67]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [68]:
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [69]:
# another way : softmax using
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
xbow3.shape

torch.Size([4, 8, 3])

In [73]:
torch.allclose(xbow2, xbow3) # they are equal

True

In [89]:
matrix = torch.tril(torch.ones(3,3))
matrix

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [92]:
matrix.transpose(0, 1)

tensor([[1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])

In [98]:
matrix_a = torch.randn((4, 6, 1))
matrix_b = torch.transpose(matrix_a, dim0=-3, dim1=-1)
matrix_c = torch.transpose(matrix_a, dim0=2, dim1=0)

In [99]:
print(f"Before transponse: {matrix_a.shape}\nAfter transpose: {matrix_b.shape}")
print(f"another way: {matrix_c.shape}")

Before transponse: torch.Size([4, 6, 1])
After transpose: torch.Size([1, 6, 4])
another way: torch.Size([1, 6, 4])


In [138]:
B, T, C = 4, 8, 32

x = torch.randn(B, T, C)

# single attention head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5
# (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril((torch.ones(T, T))) # 8 * 8 lower triangular matrix

# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
out = wei @ v
# out = wei @ x
out.shape

torch.Size([4, 8, 16])

In [139]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [140]:
wei.shape # (4, 8, 8) # attention weight for 4 in a batch

torch.Size([4, 8, 8])

In [141]:
wei[0] # masked attention weight

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5158, 0.4842, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5010, 0.2074, 0.2916, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2046, 0.3410, 0.1552, 0.2992, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3809, 0.1553, 0.1121, 0.1547, 0.1970, 0.0000, 0.0000, 0.0000],
        [0.1019, 0.1457, 0.2960, 0.1389, 0.1486, 0.1689, 0.0000, 0.0000],
        [0.1053, 0.1630, 0.1555, 0.1336, 0.1310, 0.1844, 0.1272, 0.0000],
        [0.1231, 0.1636, 0.0634, 0.1180, 0.0877, 0.1818, 0.1162, 0.1462]],
       grad_fn=<SelectBackward0>)

In [142]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)

print("Variance of k: ", k.var())
print("Variance of q: ", q.var())
print("Variance of wei: ", wei.var())

Variance of k:  tensor(0.8975)
Variance of q:  tensor(0.9451)
Variance of wei:  tensor(13.4076)


In [149]:
# after scaling
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5

print("Variance of k: ", k.var())
print("Variance of q: ", q.var())
print("Variance of wei: ", wei.var())

Variance of k:  tensor(0.9945)
Variance of q:  tensor(1.0866)
Variance of wei:  tensor(0.8950)


In [150]:
# multihead attention
# feed forward neural network
# repeat the block for multiple time
# decode the output